In [1]:
function featuresToString(tuple) {
    return tuple[0] + "," + tuple[2];
}


In [2]:

if (args.length < 5) {
    print(
        "Usage: bin/eclairjs.sh examples/mllib/als.js <ratings_file> <rank> <iterations> <output_dir> [<blocks>]");
    exit(1);
}
var sparkConf = new SparkConf().setAppName("JavaALS");
var rank = parseInt(args[2]);
var iterations = parseInt(args[3]);
var outputDir = args[4];
var blocks = -1;
if (args.length == 6) {
    blocks = parseInt(args[5]);
}

var sc = new SparkContext(sparkConf);
var lines = sc.textFile(args[1]);


In [3]:
var ratings = lines.map(function(line){
    var tok = line.split(",");
    var x = parseInt(tok[0]);
    var y = parseInt(tok[1]);
    var rating = parseFloat(tok[2]);
    return new Rating(x, y, rating);
});

var model = ALS.train(ratings, rank, iterations, 0.01, blocks);

var userFeaatureRDD = model.userFeatures()
                            .map(featuresToString)
                            .saveAsTextFile(outputDir + "/userFeatures");

var productFeaturesRDD = model.productFeatures()
                            .map(featuresToString)
                            .saveAsTextFile(outputDir + "/productFeatures");

print("Final user/product features written to " + outputDir);

sc.stop();
